In [3]:
from sympy import *

In [3]:
# do: object distance
# di: image distance
do, di, S = symbols("d_o d_i S", real=True, positive=True)
theta = symbols("theta", real=True)

In [4]:
theta + di + S + do

S + d_i + d_o + theta

In [5]:
Rl = Matrix([[cos(theta), 0, -sin(theta)], [0, 1, 0], [sin(theta), 0, cos(theta)]])
Rl  # Rotating matrix from world frame to left camera frame

Matrix([
[cos(theta), 0, -sin(theta)],
[         0, 1,           0],
[sin(theta), 0,  cos(theta)]])

In [6]:
Rr = Matrix([[cos(theta), 0, sin(theta)], [0, 1, 0], [-sin(theta), 0, cos(theta)]])
Rr  # Rotating matrix from world frame to right camera frame

Matrix([
[ cos(theta), 0, sin(theta)],
[          0, 1,          0],
[-sin(theta), 0, cos(theta)]])

In [7]:
Tl = Matrix([do * sin(theta), 0, -do * cos(theta)])
Tl  # Translating matrix from world frame to left camera frame

Matrix([
[ d_o*sin(theta)],
[              0],
[-d_o*cos(theta)]])

In [8]:
Tr = Matrix([-do * sin(theta), 0, -do * cos(theta)])
Tr  # Translating matrix from world frame to right camera frame

Matrix([
[-d_o*sin(theta)],
[              0],
[-d_o*cos(theta)]])

In [9]:
x, y, z, dx, dy, dz = symbols("x y z Delta_x Delta_y Delta_z", real=True)

Xw1 = Matrix([x, y, z])  # World coordinates of the point
Xw2 = Matrix([x + dx, y + dy, z + dz])  # World coordinates of the point
flipZ = Matrix(
    [[1, 0, 0], [0, 1, 0], [0, 0, -1]]
)  # Apply reflection matrix to flip Z direction between world frame and camera frame

In [10]:
extrinsicl = Rl.row_join(Tl)  # Extrinsic matrix from world frame to left camera frame
extrinsicr = Rr.row_join(Tr)  # Extrinsic matrix from world frame to right camera frame
Pl = flipZ * extrinsicl  # Projection matrix from world frame to left camera frame
Pr = flipZ * extrinsicr  # Projection matrix from world frame to right camera frame

In [11]:
Pl

Matrix([
[ cos(theta), 0, -sin(theta), d_o*sin(theta)],
[          0, 1,           0,              0],
[-sin(theta), 0, -cos(theta), d_o*cos(theta)]])

In [12]:
Pr

Matrix([
[cos(theta), 0,  sin(theta), -d_o*sin(theta)],
[         0, 1,           0,               0],
[sin(theta), 0, -cos(theta),  d_o*cos(theta)]])

In [13]:
Xl1 = Pl * Xw1.row_insert(3, Matrix([1]))  # Point 1 in left camera frame
Xl2 = Pl * Xw2.row_insert(3, Matrix([1]))  # Point 2 in left camera frame
Xr1 = Pr * Xw1.row_insert(3, Matrix([1]))  # Point 1 in right camera frame
Xr2 = Pr * Xw2.row_insert(3, Matrix([1]))  # Point 2 in right camera frame

In [14]:
Xol1 = (
    (do / (1 - tan(theta) * Xl1[0] / Xl1[2])) / (Xl1[2]) * Xl1
)  # Point 1 in left image frame object plane
Xol2 = (
    (do / (1 - tan(theta) * Xl2[0] / Xl2[2])) / (Xl2[2]) * Xl2
)  # Point 2 in left image frame object plane
Xor1 = (
    (do / (1 - tan(theta) * Xr1[0] / Xr1[2])) / (Xr1[2]) * Xr1
)  # Point 1 in right image frame object plane
Xor2 = (
    (do / (1 - tan(theta) * Xr2[0] / Xr2[2])) / (Xr2[2]) * Xr2
)  # Point 2 in right image frame object plane

In [15]:
simplify(Xol1.subs({theta: 0, z: 0}))

Matrix([
[  x],
[  y],
[d_o]])

In [16]:
simplify(Xol2.subs({theta: 0, z: 0}))

Matrix([
[-d_o*(Delta_x + x)/(Delta_z - d_o)],
[-d_o*(Delta_y + y)/(Delta_z - d_o)],
[                               d_o]])

In [17]:
simplify(Xor1.subs({theta: 0, z: 0}))

Matrix([
[  x],
[  y],
[d_o]])

In [18]:
simplify(Xor2.subs({theta: 0, z: 0}))

Matrix([
[-d_o*(Delta_x + x)/(Delta_z - d_o)],
[-d_o*(Delta_y + y)/(Delta_z - d_o)],
[                               d_o]])

In [19]:
dXol = Xol2 - Xol1  # Distance in left image frame object plane
dXor = Xor2 - Xor1  # Distance in right image frame object plane

In [20]:
simplify(dXol.subs({theta: 0, z: 0, dz: 0}))

Matrix([
[Delta_x],
[Delta_y],
[      0]])

In [21]:
simplify(dXor.subs({theta: 0, z: 0, dz: 0}))

Matrix([
[Delta_x],
[Delta_y],
[      0]])

# $\alpha = 0$ condition

In [22]:
Xil1 = -di / Xol1[2] * Xol1  # Point 1 in left image frame image plane
Xil2 = -di / Xol2[2] * Xol2  # Point 2 in left image frame image plane
Xir1 = -di / Xor1[2] * Xor1  # Point 1 in right image frame image plane
Xir2 = -di / Xor2[2] * Xor2  # Point 2 in right image frame image plane

In [23]:
dXil = Xil2 - Xil1  # Distance in left image frame image plane
dXir = Xir2 - Xir1  # Distance in right image frame image plane

In [24]:
simplify(dXil.subs({theta: 0, z: 0, dz: 0}))

Matrix([
[-Delta_x*d_i/d_o],
[-Delta_y*d_i/d_o],
[               0]])

In [25]:
simplify(dXir.subs({theta: 0, z: 0, dz: 0}))

Matrix([
[-Delta_x*d_i/d_o],
[-Delta_y*d_i/d_o],
[               0]])

In [26]:
simplify(dXil[0])

d_i*(-Delta_x*d_o + Delta_x*z - Delta_z*x)/((-d_o*cos(theta) + x*sin(theta) + z*cos(theta))*(-d_o*cos(theta) + (Delta_x + x)*sin(theta) + (Delta_z + z)*cos(theta)))

In [27]:
simplify(dXir[0])

d_i*(-Delta_x*d_o + Delta_x*z - Delta_z*x)/((d_o*cos(theta) + x*sin(theta) - z*cos(theta))*(d_o*cos(theta) + (Delta_x + x)*sin(theta) - (Delta_z + z)*cos(theta)))

In [28]:
simplify(dXil[1])

d_i*(-Delta_x*y*sin(theta) - Delta_y*d_o*cos(theta) + Delta_y*x*sin(theta) + Delta_y*z*cos(theta) - Delta_z*y*cos(theta))/((-d_o*cos(theta) + x*sin(theta) + z*cos(theta))*(-d_o*cos(theta) + (Delta_x + x)*sin(theta) + (Delta_z + z)*cos(theta)))

In [29]:
simplify(dXir[1])

d_i*(Delta_x*y*sin(theta) - Delta_y*d_o*cos(theta) - Delta_y*x*sin(theta) + Delta_y*z*cos(theta) - Delta_z*y*cos(theta))/((d_o*cos(theta) + x*sin(theta) - z*cos(theta))*(d_o*cos(theta) + (Delta_x + x)*sin(theta) - (Delta_z + z)*cos(theta)))

In [30]:
n = 3
dxil_val = symbols("delta_il1:%d" % (n + 1))  # Distance in left image frame image plane
dxir_val = symbols(
    "delta_ir1:%d" % (n + 1)
)  # Distance in right image frame image plane
dxil_val, dxir_val

((delta_il1, delta_il2, delta_il3), (delta_ir1, delta_ir2, delta_ir3))

In [31]:
sol_dx_dz = solve(
    (
        Eq(dXil[0], dxil_val[0]),
        Eq(dXir[0], dxir_val[0]),
    ),
    (dx, dz),
)

In [32]:
sol_dx_dz  # 无解，不符合沙姆定律😭

[]

In [33]:
simplify(Eq(dXil[0], dxil_val[0]).subs({theta: pi / 4, z: 0, di: 1, do: 1, x: 0}))

Eq(delta_il1, 2*Delta_x/(Delta_x + Delta_z - 1))

In [34]:
simplify(Eq(dXir[0], dxir_val[0]).subs({theta: pi / 4, z: 0, di: 1, do: 1, x: 0}))

Eq(delta_ir1, -2*Delta_x/(Delta_x - Delta_z + 1))

In [35]:
sol_dx = sol_dx_dz[dx]
sol_dz = sol_dx_dz[dz]

TypeError: list indices must be integers or slices, not Symbol

In [ ]:
sol_dy1 = solve((Eq(dXil[1], dxil_val[1])), dy)

sol_dy2 = solve((Eq(dXir[1], dxir_val[1])), dy)

sol_dy = (sol_dy1[dy] + sol_dy2[dy]) / 2

In [ ]:
simplify(sol_dx.subs({theta: 0, z: 0, dz: 0}))

(-S*dx_il1/2 - S*dx_ir1/2 + dx_il1*x - dx_ir1*x)/(Mn*S - dx_il1 + dx_ir1)

In [ ]:
simplify(sol_dy.subs({theta: 0, z: 0, dz: 0}))

(-2*Delta_z*Mn*y + Delta_z*dx_il2 + Delta_z*dx_ir2 - d_o*dx_il2 - d_o*dx_ir2)/(2*Mn*d_o)

In [ ]:
simplify(sol_dz.subs({theta: 0, z: 0, dz: 0}))

d_o*(-dx_il1 + dx_ir1)/(Mn*S - dx_il1 + dx_ir1)

In [ ]:
print(latex(sol_dx_dz))
print(latex(sol_dy))

\left\{ \Delta_{x} : \frac{- \frac{S d_{o} dx_{il1}}{2} - \frac{S d_{o} dx_{ir1}}{2} + \frac{S dx_{il1} z}{2} + \frac{S dx_{ir1} z}{2} + d_{o} dx_{il1} x - d_{o} dx_{ir1} x - dx_{il1} x z + dx_{ir1} x z}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}, \  \Delta_{z} : \frac{- d_{o}^{2} dx_{il1} + d_{o}^{2} dx_{ir1} + 2 d_{o} dx_{il1} z - 2 d_{o} dx_{ir1} z - dx_{il1} z^{2} + dx_{ir1} z^{2}}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}\right\}
\frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{il2} + \Delta_{z} dx_{il2} z + d_{o}^{2} dx_{il2} - 2 d_{o} dx_{il2} z + dx_{il2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)} + \frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{ir2} + \Delta_{z} dx_{ir2} z + d_{o}^{2} dx_{ir2} - 2 d_{o} dx_{ir2} z + dx_{ir2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)}


$$
\left\{ \Delta_{x} : \frac{- \frac{S d_{o} dx_{il1}}{2} - \frac{S d_{o} dx_{ir1}}{2} + \frac{S dx_{il1} z}{2} + \frac{S dx_{ir1} z}{2} + d_{o} dx_{il1} x - d_{o} dx_{ir1} x - dx_{il1} x z + dx_{ir1} x z}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}, \  \Delta_{z} : \frac{- d_{o}^{2} dx_{il1} + d_{o}^{2} dx_{ir1} + 2 d_{o} dx_{il1} z - 2 d_{o} dx_{ir1} z - dx_{il1} z^{2} + dx_{ir1} z^{2}}{Mn S d_{o} - d_{o} dx_{il1} + d_{o} dx_{ir1} + dx_{il1} z - dx_{ir1} z}\right\}
$$

$$
\Delta_{y}: \frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{il2} + \Delta_{z} dx_{il2} z + d_{o}^{2} dx_{il2} - 2 d_{o} dx_{il2} z + dx_{il2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)} + \frac{\Delta_{z} Mn d_{o} y - \Delta_{z} d_{o} dx_{ir2} + \Delta_{z} dx_{ir2} z + d_{o}^{2} dx_{ir2} - 2 d_{o} dx_{ir2} z + dx_{ir2} z^{2}}{2 \left(- Mn d_{o}^{2} + Mn d_{o} z\right)}
$$